In [1]:
import torch
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torchvision import models

from scipy.special import softmax
from util.infer_stitch import InferDiv, InferStitch

import numpy as np
import os
from PIL import Image
import csv
import gc

ModuleNotFoundError: No module named 'torch'

In [ ]:
"""------------------------ GPU Set ---------------------------"""
gpu_device = '0' # GPU 번호
"""------------------------ Model Set ---------------------------"""
model_mode = 'DeepFCN101' # DeepRes101 / DeepRes50 / DeepFCN101

In [ ]:
data_dir = '/Projects/AISpark_Challenge_IRDIS/Data/AIHub/Validation'
img_folder_name = 'vaildate_buildings_data'
label_folder_name = 'vaildate_buildings_labeling'
gt_format = '.png'
input_size = 1024 # 학습으로 들어가는 영상 크기
original_size = 1024 # 데이터셋 영상 크기
x_window_step = 200
y_window_step = 200
BACKBONE_DIR = '/Projects/AISpark_Challenge_IRDIS/Data/AIHub/weight/building/'
BACKBONE_NAME = '1024by1024_multiGPU_weights/model_deepFCN101_ep186.pth'# woojun0327.pth, #resunet50_149.pth' #fcnres101_ep67.pth, resunet101_ep149.pth #1024by1024_multiGPU_weights/model_deepFCN101_ep186.pth, #1024by1024_multiGPU_weights/model_deepRES50_ep169.pth
csv_data = '/Projects/AISpark_Challenge_IRDIS/Data/AIHub/Validation/vaildate_buildings.csv'

In [ ]:
"""------------------------ SAVE ---------------------------"""
RESULTS_DIR = '/Module 1/result' # 저장 경로
"""------------------------ function parameters ---------------------------"""
num_classes = 7  # 0:배경, 1:소형, 2:아파트, 3:공장, 4:중형단독시설, 5:대형시설, 6:contour
trans_ratio = 0.5 # save 투영율 (1일수록 짙은 label)

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES']= gpu_device
IMG_MEAN = np.array((128, 128, 128), dtype=np.float32) # 학습 속도를 위해 RGB 128을 영점으로 둔다. [-128~127], Load code에서 128로 나눔 [-1~0.999]
INPUT_SIZE_m = [input_size, input_size]
original_size = [original_size, original_size]  # width, height

In [ ]:
def main():
    cudnn.enabled = True

    """-------------------------- 개발 MODEL ROAD --------------------------"""
    if model_mode == 'DeepRes101':
        model = models.segmentation.deeplabv3_resnet101(pretrained=False, num_classes=num_classes)
    elif model_mode == 'DeepRes50':
        model = models.segmentation.deeplabv3_resnet50(pretrained=False, num_classes=num_classes)
    elif model_mode == 'DeepFCN101':
        model = models.segmentation.fcn_resnet101(pretrained=False, num_classes=num_classes)
    else:
        raise Exception("Please select a model")

    model.cuda(0)
    # model=nn.DataParallel(model)
    model.load_state_dict(torch.load(BACKBONE_DIR + BACKBONE_NAME),strict=False)
    #model.load_state_dict(torch.load('model_deepFCN101_ep186.pth'))

    #메모리 부족 수정 부분
    device = torch.device("cuda")
    model.to(device)
    #메모리 부분 수정 부분
    torch.cuda.memory_summary(device=None, abbreviated=False)

    model.eval()
    cudnn.benchmark = False  # cudnn.benchmark = true -- uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms.

    """-------------------------- FILE LOAD ----------------------------------"""
    name_list = []
    f = open(csv_data, 'r')
    names = csv.reader(f)
    for name in names:
        name[0] = name[0] + gt_format
        name_list.append(name[0])
    f.close()

    divided_img = InferDiv(size_sample=INPUT_SIZE_m, original_size=original_size, x_window_step=x_window_step, y_window_step=y_window_step)
    stitching_img = InferStitch(size_sample=INPUT_SIZE_m, size_output=original_size)


    """-------------------------- TEST START ----------------------------------"""
    for cntList in range(name_list.__len__()):

        # image open
        img_file = data_dir + "/"+ img_folder_name +"/%s" % name_list[cntList]
        img_rgb = Image.open(img_file).convert('RGB')

        ''' 영상 crop 해서 계산 한 후 다시 붙여 주는 부분이 필요 (image 만 계산) '''
        imgset, num_patches = divided_img.forward(img_rgb, IMG_MEAN) # 영상을 crop size 로 쪼개는 부분

        for cnt_patches in range(num_patches):
            onepatch = imgset[cnt_patches]['divided_img']
            onepatch = onepatch.expand(1, 3, INPUT_SIZE_m[0], INPUT_SIZE_m[1])
            if cnt_patches == 0:
                sets = onepatch
            else:
                sets = torch.cat((sets, onepatch), 0)

        image_input = Variable(sets).cuda(0)

        ''' 모델 연산 '''
        with torch.no_grad():
            pred_model = model(image_input).to(torch.device('cuda'))
            pred_model = pred_model['out']
            gc.collect()
            torch.cuda.empty.cache()

        pred_model = pred_model.cpu().detach().numpy()


        ''' output 결과들을 병합 '''
        results = stitching_img.forward(pred_model, imgset, num_classes=num_classes)
        results = softmax(results, axis=0)

        if results.shape[0] > 6: # conture 제거 작업
            con_idx_i, con_idx_j = np.where(results[6,:,:] >= 0.5)
            results[0,con_idx_i, con_idx_j] = 1 # conture 위치에 1을 주어서 배경으로 바꿈
            results = results[0:-1,:,:]

        results = np.expand_dims(results, axis=0)
        results = torch.tensor(results)
        results = Variable(results)

        """-------------------------- RESULT SAVE ----------------------------------"""
        results = results.cpu().detach().numpy()
        results = softmax(results, axis=1)
        np.savez(RESULTS_DIR  + 'featuremap/' +'GT_' + name_list[cntList])

        print('progress: ', cntList, '/', name_list.__len__())


In [ ]:
main()